# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import re
import pickle
import numpy as np


import pandas as pd
from sqlalchemy import create_engine

from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [2]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table("InsertTableName",engine)

In [4]:
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [5]:
# Inspecting the count of every category
df.iloc[:,4:].sum()

related                   20298.0
request                    4480.0
offer                       119.0
aid_related               10878.0
medical_help               2087.0
medical_products           1314.0
search_and_rescue           724.0
security                    471.0
military                    860.0
child_alone                   0.0
water                      1674.0
food                       2930.0
shelter                    2319.0
clothing                    406.0
money                       604.0
missing_people              299.0
refugees                    876.0
death                      1196.0
other_aid                  3448.0
infrastructure_related     1705.0
transport                  1203.0
buildings                  1335.0
electricity                 534.0
tools                       159.0
hospitals                   283.0
shops                       120.0
aid_centers                 309.0
other_infrastructure       1151.0
weather_related            7302.0
floods        

In [6]:
# dropping the 'child_alone'-category, since it has just zeros and the 'original'-column since we're only interested in the english text
df.drop(['child_alone', 'original'], axis=1, inplace=True)

In [7]:
# Inspecting for nans
pd.isnull(df).sum()

id                            0
message                       0
original                  16122
genre                         0
related                     138
request                     138
offer                       138
aid_related                 138
medical_help                138
medical_products            138
search_and_rescue           138
security                    138
military                    138
water                       138
food                        138
shelter                     138
clothing                    138
money                       138
missing_people              138
refugees                    138
death                       138
other_aid                   138
infrastructure_related      138
transport                   138
buildings                   138
electricity                 138
tools                       138
hospitals                   138
shops                       138
aid_centers                 138
other_infrastructure        138
weather_

In [8]:
# Because all categorgies have the same number of nans, we should look at some datapoints where nan values exist
df[pd.isnull(df)].head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# It seems that if a category has a nan-value, all of the other columns are nans as well, hence they can be dropped
df.dropna(inplace=True)

In [10]:
# Split into X and y
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [11]:
# Taken from the Grid Search Pipeline-Solution
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    stop_words = stopwords.words("english")
    
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)

    clean_tokens = [WordNetLemmatizer().lemmatize(w) for w in tokens if w not in stop_words]
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(RandomForestClassifier()))
                ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
y_pred = pipeline.predict(X_test)
for i, col in enumerate(y_test.columns):
    print(f"{col}: \n")
    print(classification_report(y_test.values[:,i] , y_pred[:,i]))



related: 

             precision    recall  f1-score   support

        0.0       0.36      0.21      0.26       850
        1.0       0.66      0.81      0.73      1675
        2.0       0.17      0.03      0.05        31

avg / total       0.55      0.60      0.57      2556

request: 

             precision    recall  f1-score   support

        0.0       0.67      0.89      0.76      1680
        1.0       0.40      0.14      0.21       876

avg / total       0.57      0.63      0.57      2556

offer: 

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      2554
        1.0       0.00      0.00      0.00         2

avg / total       1.00      1.00      1.00      2556

aid_related: 

             precision    recall  f1-score   support

        0.0       0.63      0.85      0.73      1569
        1.0       0.47      0.21      0.29       987

avg / total       0.57      0.60      0.56      2556

medical_help: 

             precision   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
parameters = {'clf__estimator__max_features': ["auto", "log2"],
              'clf__estimator__n_estimators': [10, 50]}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2, n_jobs=-1)
cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__max_features=auto, clf__estimator__n_estimators=10 
[CV]  clf__estimator__max_features=auto, clf__estimator__n_estimators=10, total=  13.9s
[CV] clf__estimator__max_features=auto, clf__estimator__n_estimators=10 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   18.3s remaining:    0.0s


[CV]  clf__estimator__max_features=auto, clf__estimator__n_estimators=10, total=  14.0s
[CV] clf__estimator__max_features=auto, clf__estimator__n_estimators=10 
[CV]  clf__estimator__max_features=auto, clf__estimator__n_estimators=10, total=  14.1s
[CV] clf__estimator__max_features=auto, clf__estimator__n_estimators=50 
[CV]  clf__estimator__max_features=auto, clf__estimator__n_estimators=50, total=  47.5s
[CV] clf__estimator__max_features=auto, clf__estimator__n_estimators=50 
[CV]  clf__estimator__max_features=auto, clf__estimator__n_estimators=50, total=  47.4s
[CV] clf__estimator__max_features=auto, clf__estimator__n_estimators=50 
[CV]  clf__estimator__max_features=auto, clf__estimator__n_estimators=50, total=  48.5s
[CV] clf__estimator__max_features=log2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__max_features=log2, clf__estimator__n_estimators=10, total=  14.3s
[CV] clf__estimator__max_features=log2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__max_features

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  7.7min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__max_features': ['auto', 'log2'], 'clf__estimator__n_estimators': [10, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [34]:
temp = classification_report(y_test.values[:,i] , y_pred[:,i])

In [39]:
y_pred

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  1.,  0., ...,  0.,  0.,  0.]])

In [45]:
#y_pred = cv.predict(X_test)
acc = []
precision = []
recall = []

for i, col in enumerate(y_test.columns):
    acc.append(accuracy_score(y_test.values[i,:], y_pred[i,:]))
    precision.append(precision_score(y_test.values[i,:], y_pred[i,:]))
    recall.append(recall_score(y_test.values[i,:], y_pred[i,:]))

print(f"Accuracy: {np.mean(acc):.2}")
print(f"Precision: {np.mean(precision):.2}")
print(f"Recall: {np.mean(recall):.2}")

Accuracy: 0.94
Precision: 0.5
Recall: 0.31


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [46]:
pickle.dump(cv, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.